In [97]:
import numpy as np

In [170]:
def E_P(stock_price,security_price, interest, vol, K, T, simulation_num, barrier,return_rate, dividend_rate):
    # 함수 하나로 price가 바로 나올 수 있게 simulation 수도 함수 안에 포함해서 simulation을 한 방에 실행
    dt = 1/252
    payoff = [security_price*(1+return_rate*i/2) for i in range(1,7)] # 조기상환 시에 payoff가 정해져있기 때문에 list로 미리 만듦
    num=[0,0,0,0,0,0,0] # 각 조건에서의 횟수를 담을 list
    s0 = stock_price
    r = interest
    sigma = vol
    d = dividend_rate
    discount = [np.exp(-(r-d)*T*i/len(K)) for i in range(1,7)] #[exp(-r/2), exp(-r), ... , exp(-3r)]
    ep_list= []
    t = T*252
    ki_stock_sum = 0
    np.random.seed(0)
    for i in range(simulation_num):
        s = s0 #초기화
        s_path=[]
        for k in range(t+1):

            ep = np.random.standard_normal()
            ep_list.append(ep)
            
            s = s * np.exp((r-0.5*sigma**2)*dt + sigma*ep*np.sqrt(dt))
            s_path.append(s)
            
        if s_path[int(t/6)] >= s0 * K[0]: #252는 6의 배수임
            num[0] += 1

        elif s_path[int(t/3)] >= s0 * K[1]:
            num[1] += 1

        elif s_path[int(t/2)] >= s0 * K[2]:
            num[2] += 1

        elif s_path[int(t*2/3)] >= s0 * K[3]:
            num[3] += 1

        elif s_path[int(t*5/6)] >= s0 * K[4]:
            num[4] += 1

        
        elif min(s_path) <= s0 * barrier:
            
            if s_path[int(t)] >= s0 * K[5]:
                    num[5] += 1
                    
            else:
                ki_stock_sum = ki_stock_sum + s
                num[6] += 1
    ki_stock_ex = ki_stock_sum/num[6]
    loss_ratio = ki_stock_ex/s0
    
    weight = [num[i]/simulation_num*discount[i] for i in range(6)]
    weight.append(num[6]*discount[5]/simulation_num)
    result = [weight[i]*payoff[i] for i in range(6)]
    result.append(weight[6]*security_price*loss_ratio)
    ELS_price = sum(result)
    
    
    return ELS_price


In [131]:
# 각 parameter들 설정
stock_price = 726000 # 23.5.8자 LG화학 종가, 계약서에서 종가를 사용한다고 명시하였기 때문에 종가를 사용, Q. 보통 종가를 사용하나 ?
security_price = 10000
interest = np.log(1.034) # 금투협 채권정보센터 나이스 피앤아이 5.8일자 1년물 채권의 수익률
vol = 0.4217
K = np.array([0.9,0.85,0.8,0.8,0.75,0.75]) # 각 조기행사 조건
T = 3 # 23~26년까지 3년
simulation_num = 1000 # 몇 번이 최적의 simul 횟수일까 ?
barrier = 0.4 # ELS의 가치가 없어지는 지점. 40%로 설정
return_rate = 0.1136 # 연 수익률 11.36%
dividend_rate = np.log(1.0147)# 연 배당수익률 1.47%

In [171]:
price = E_P(stock_price, security_price, interest, vol, K, T, simulation_num, barrier, return_rate, dividend_rate)
print(price)

# 시뮬레이션이 잘못돌아가고 있음, 한 쪽으로 몰리는 현상이 생김

([6218.531096291467, 1398.8022220502899, 682.6706724118831, 413.635337571692, 330.72847774358087, 152.05356683899808, 401.71883552098546], 9598.140208428898)
